<a href="https://colab.research.google.com/github/PurvishParekh/FCC_Neural_Network_SMS_Classifier/blob/main/Neural_Network_SMS_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip uninstall -y tf-nightly
# !pip install tensorflow
# except Exception:
#   pass
%tensorflow_version 2.x
import tensorflow as tf
import pandas as pd
# from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

print(dir(tf.keras))

['Input', 'Model', 'Sequential', '__builtins__', '__cached__', '__doc__', '__file__', '__internal__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'activations', 'applications', 'backend', 'callbacks', 'constraints', 'datasets', 'dtensor', 'estimator', 'experimental', 'export', 'initializers', 'layers', 'losses', 'metrics', 'mixed_precision', 'models', 'optimizers', 'preprocessing', 'regularizers', 'saving', 'utils']


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-07-07 13:15:56--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2024-07-07 13:15:56 (9.59 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-07-07 13:15:56--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
train_file = pd.read_csv(train_file_path, sep='\t', header=None, names=['category', 'message'])
test_file = pd.read_csv(test_file_path, sep='\t',  header=None, names=['category', 'message'])

In [ ]:
from string import digits, punctuation
remove_set = digits + punctuation
words = {word.lower().strip(remove_set) for line in train_file['message']
         for word in line.rsplit(",",1)[0].split()}

In [ ]:
len(words)

6732

In [ ]:
train_file.tail()

,category,message
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...
4178,ham,tell my bad character which u dnt lik in me. ...


In [ ]:
test_file.tail()

,category,message
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...
1391,spam,not heard from u4 a while. call 4 rude chat pr...


In [ ]:
train_labels = train_file.pop('category')
test_labels = test_file.pop('category')


In [ ]:
train_labels.head()

0    ham
1    ham
2    ham
3    ham
4    ham
Name: category, dtype: object

In [ ]:
train_labels = pd.factorize(train_labels)[0]
test_labels = pd.factorize(test_labels)[0]

In [ ]:
train_labels

array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_file['message'].values, train_labels)
)
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_file['message'].values, test_labels)
)

In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
vec = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))

In [ ]:
vocab = np.array(vec.get_vocabulary())

In [ ]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(1e-4),metrics=['acc'])


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 1000)              0         
 ctorization)                                                    
                                                                 
 embedding_4 (Embedding)     (None, 1000, 64)          64000     
                                                                 
 bidirectional_8 (Bidirecti  (None, 1000, 128)         66048     
 onal)                                                           
                                                                 
 bidirectional_9 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                      

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)

Epoch 1/10
131/131 [==============================] - 46s 233ms/step - loss: 0.6035 - acc: 0.8634 - val_loss: 0.4833 - val_acc: 0.8604
Epoch 2/10
131/131 [==============================] - 20s 155ms/step - loss: 0.3765 - acc: 0.8662 - val_loss: 0.2514 - val_acc: 0.8698
Epoch 3/10
131/131 [==============================] - 21s 159ms/step - loss: 0.1733 - acc: 0.9519 - val_loss: 0.1232 - val_acc: 0.9677
Epoch 4/10
131/131 [==============================] - 20s 151ms/step - loss: 0.0955 - acc: 0.9785 - val_loss: 0.0757 - val_acc: 0.9812
Epoch 5/10
131/131 [==============================] - 20s 153ms/step - loss: 0.0653 - acc: 0.9844 - val_loss: 0.0644 - val_acc: 0.9844
Epoch 6/10
131/131 [==============================] - 20s 155ms/step - loss: 0.0522 - acc: 0.9873 - val_loss: 0.0599 - val_acc: 0.9844
Epoch 7/10
131/131 [==============================] - 19s 147ms/step - loss: 0.0433 - acc: 0.9895 - val_loss: 0.0552 - val_acc: 0.9865
Epoch 8/10
131/131 [==============================] - 2

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict([pred_text])
  x = float(pred[0][0])
  if x < 0.5:
    prediction = [x, 'ham']
  else:
    prediction = [x, 'spam']

  return (prediction)

pred_text = "how are you doing today?"
pred_text2 = "sale today! to stop texts call 98912460324"
prediction = predict_message(pred_text)
pred2 = predict_message(pred_text2)
print(prediction)
print(pred2)

1/1 [==============================] - 0s 33ms/step
[0.0010851181577891111, 'ham']
[0.7173376679420471, 'spam']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 39ms/step
You passed the challenge. Great job!
